# Property Calculation with Cubic EoS

This workbook shows examples of calculation of physical properties for pure components and mixtures using a cubic equation of state model (Peng-Robinson EoS ``preos`` is used in these examples).

**Note:** Thermal derived properties (residual entropy, residual enthalpy, residual heat capacities and speed of sound) need partial temperature derivatives, which is computed numerically using the classical 4th order Runge–Kutta method ($O(h^4)$ approximation).

# Pure component properties

In [11]:
import numpy as np
from phasepy import component, preos
water = component(name='water', Tc=647.13, Pc=220.55, Zc=0.229, Vc=55.948, w=0.344861,
                  GC={'H2O':1}, Mw=18.04)
eos = preos(water)

## Density

The density of the fluid is computed with the ``eos.density`` method [mol/cm3]. It requires the temperature, pressure and the aggregation state.

In [12]:
T = 340.0 # K
P = 1.0 # bar
eos.density(T, P, 'L'), eos.density(T, P, 'V')

(0.045697369422639834, 3.5769380018112745e-05)

## Pressure

The pressure of the fluid can be computed at given molar volume and temperature using the ``eos.pressure`` method [bar]. 

In [13]:
rhol = 0.045697369422639834
vl = 1.0/rhol
rhov = 3.5769380018112745e-05
vv = 1.0/rhov
eos.pressure(T, vl), eos.pressure(T, vv)

(array([1.]), array([1.]))

## Validation of phase equilibrium

For pure fluids, the ``eos.psat`` method allows to compute the saturation pressure at given temperature. It returns the equilibrium pressure and molar volumes of the liquid and vapor phase. 

The phase equilibria can be verified through fugacity coefficients using the ``eos.logfug`` method or by using chemical potentials with the ``eos.muad`` method.

In [14]:
Psat, vlsat, vvsat = eos.psat(T)

# Get fugacity coefficients and phase volumes
logfugl, vlsat = eos.logfug(T, Psat, 'L')
logfugv, vvsat = eos.logfug(T, Psat, 'V')

# Calculate chemical potentials
Tfactor, Pfactor, rofactor, tenfactor, zfactor = eos.sgt_adim(T)
Tad = T * Tfactor
rholad = 1/vlsat * rofactor
rhovad = 1/vvsat * rofactor
mul = eos.muad(rholad, Tad)
muv = eos.muad(rhovad, Tad)

print('Fugacity coefficients:', logfugl, logfugv, 'are equal:', np.allclose(logfugl, logfugv))
print('Chemical potentials:', mul, muv, 'are equal:', np.allclose(mul, muv))

Fugacity coefficients: [-0.00269341] [-0.00269341] are equal: True
Chemical potentials: [-0.61292691] [-0.61292691] are equal: True


## Thermal derived properties

The ``eos`` object also includes the calculation of some thermal derived properties such as residual entropy (``eos.EntropyR``), residual enthalpy (``eos.EnthalpyR``), residual isochoric heat capacity (``eos.CvR``), and residual isobaric heat capacity (``eos.CpR``).

For the speed of sound calculation (``eos.speed_sound``) the ideal gas heat capacities are required. In the example the isochoric and isobaric ideal gas contribution are set to $3R/2$ and $5R/2$, respectively. Better values of ideal gas heat capacities contribution can be found from empirical correlations, e.g. DIPPR 801.

In [15]:
# vaporization entropy [J/(mol K)]
Svap = eos.EntropyR(T, Psat, 'V') - eos.EntropyR(T, Psat, 'L')

# vaporization enthalpy [J/mol]
Hvap = eos.EnthalpyR(T, Psat, 'V') - eos.EnthalpyR(T, Psat, 'L')

# isochoric and isobaric residual heat capacities [J/(mol K)]
cvr = eos.CvR(T, P, 'L')
cpr = eos.CpR(T, P, 'L')

# speed of sound using ideal gas heat capacities [m/s]
r = 8.314  # J/(mol K)
CvId = 3*r/2
CpId = 5*r/2
w = eos.speed_sound(T, P, 'V', CvId=CvId, CpId=CpId)

print('Vaporization Entropy:', Svap, 'J/(mol K)')
print('Vaporization Enthalpy:', Hvap, 'J/mol')
print('Residual isochoric heat capacity:', cvr, 'J/(mol K)')
print('Residual isobaric heat capacity:', cpr, 'J/(mol K)')
print('Speed of sound:', w, 'm/s')

Vaporization Entropy: [128.64101949] J/(mol K)
Vaporization Enthalpy: [43737.94662499] J/mol
Residual isochoric heat capacity: [30.75238107] J/(mol K)
Residual isobaric heat capacity: [47.74395699] J/(mol K)
Speed of sound: [508.71361585] m/s


# Mixture properties

Following mixture examples use the Peng-Robinson EoS using the MHV mixing rule and the UNIFAC activity coefficient model.

In [16]:
from phasepy import mixture
water = component(name='water', Tc=647.13, Pc=220.55, Zc=0.229, Vc=55.948, w=0.344861,
                  GC={'H2O':1}, Mw=18.04)
ethanol = component(name='ethanol', Tc=514.0, Pc=61.37, Zc=0.241, Vc=168.0, w=0.643558,
                    GC={'CH3':1, 'CH2':1, 'OH(P)':1}, Mw=46.07)
mix = mixture(water, ethanol)
mix.unifac()
eos = preos(mix, 'mhv_unifac')

## Density

The density of the fluid is computed with the ``eos.density`` method. It requires the composition, temperature, pressure and the aggregation state.

In [17]:
T = 340.  # K
P = 1.  # bar
x = np.array([0.3, 0.7])
eos.density(x, T, P, 'L'), eos.density(x, T, P, 'V')

(0.01905921137858399, 3.611599060036928e-05)

## Pressure

The pressure of the fluid mixture can be computed at given composition, molar volume and temperature using the ``eos.pressure`` method. 

In [18]:
rhol = eos.density(x, T, P, 'L')
vl = 1.0/rhol
rhov = eos.density(x, T, P, 'V')
vv = 1.0/rhov
eos.pressure(x, vl, T), eos.pressure(x, vv, T)

(0.999999999998181, 0.9999999999999996)

## Validation of phase equilibrium

The effective fugacity coefficients can be computed at given composition, temperature, pressure and aggregation state using the ``eos.logfugef`` method. This function returns the natural logarithm of the fugacity coefficients and the computed volume fraction. The fugacity of the mixture can be computed similarly with the ``eos.logfugmix`` method.

The chemical potential are computed at given dimensionless density vector ($\rho_i = x_i \rho b_0$) and temperature.


In [19]:
lnphi, v = eos.logfugef(x, T, P, 'L')
lnphimix, v = eos.logfugmix(x, T, P, 'L')
print('Fugacities:', lnphimix, np.dot(lnphi, x), 'are equal:', np.allclose(lnphimix, np.dot(lnphi, x)))

rhofactor = eos.b[0]
rhoad = 1/v * rofactor
rhovector = x * rhoad
print('Chemical potentials:', eos.muad(rhovector, T))

Fugacities: -0.49475747839407447 -0.49475747839408146 are equal: True
Chemical potentials: [-0.05741901  0.01584424]


## Thermal derived properties

The ``eos`` object also includes the calculation of some thermal derived properties such as residual entropy (``eos.EntropyR``), residual enthalpy (``eos.EnthalpyR``), residual isochoric heat capacity (``eos.CvR``), and residual isobaric heat capacity (``eos.CpR``).

For the speed of sound calculation (``eos.speed_sound``) the ideal gas heat capacities are required. In the example the isochoric and isobaric ideal gas contribution are set to $3R/2$ and $5R/2$, respectively. Better values of ideal gas heat capacities contribution can be found from empirical correlations, e.g. DIPPR 801.

In [20]:
Sr = eos.EntropyR(x, T, P, 'L')
Hr = eos.EnthalpyR(x, T, P, 'L')
Cvr = eos.CvR(x, T, P, 'L')
Cpr = eos.CpR(x, T, P, 'L')

# speed of sound using ideal gas heat capacities [m/s]
r = 8.314  # J / mol K
CvId = 3*r/2
CpId = 5*r/2
w = eos.speed_sound(x, T, P, 'V', CvId=CvId, CpId=CpId)

print('Residual Entropy:', Sr, 'J/(mol K)')
print('Residual Enthalpy:', Hr, 'J/mol')
print('Residual isochoric heat capacity:', Cvr, 'J/(mol K)')
print('Residual isobaric heat capacity:', Cpr, 'J/(mol K)')
print('Speed of sound:', w, 'm/s')

Residual Entropy: -116.82399490452899 J/(mol K)
Residual Enthalpy: -41118.71891716509 J/mol
Residual isochoric heat capacity: 49.63465165531006 J/(mol K)
Residual isobaric heat capacity: 80.144782348063 J/(mol K)
Speed of sound: 351.21516609921684 m/s
